# Декораторы

**Декоратор** — структурный шаблон проектирования, предназначенный для динамического подключения дополнительного поведения к объекту. Шаблон Декоратор предоставляет гибкую альтернативу практике создания подклассов с целью расширения функциональности.



Python предусматривает такой же механизм для функций:

In [ ]:
class A:
    @staticmethod
    def p():
        print("Привет, я в декораторе")

In [ ]:
A.p()

## Несколько фактов о функциях

В этом блокноте используются идеи, описанные в статьях:
- https://habr.com/ru/post/141411/
- https://habr.com/ru/post/141501/

### 1. Функции - это объекты

В первую очередь надо помнить о том, что в питоне всё - даже функции - является объектами.

In [ ]:
def p1(s="привет"):
    return s.upper()
 
print(p1())

Создадим еще одну переменную, которая будет ссылаться на ту же функцию:

In [ ]:
p2 = p1
 
print(p2())

При этом если мы удалим первую переменную-ссылку на эту функцию, то сама функция из памяти не удалится, и вторая переменная продолжит ссылаться на нее:

In [ ]:
del p1
p1()

In [ ]:
p2()

### 2. Функции можно определять внутри других функций

In [ ]:
def a():
    def b():
        print("Функция b")
    
    print("Функция a")
    b()

a()

Но стоит помнить об области видимости функции b:

In [ ]:
b()

### 3. Функции могут возвращать функции

как и любой другой объект

In [ ]:
def a():
    def b():
        print("Функция b")
    
    return b

a()

In [ ]:
c = a()
c()

In [ ]:
a()()

### 4. Callbacks

Мы можем передавать функцию как аргумент в другую функцию. Например, чтобы выполнить ее по завершении какого-то кода. Это частая практика для асинхронных приложений, когда мы не знаем точно, когда закончит выполняться один блок кода и нам надо будет выполнить следующий код.

In [ ]:
def a(callback):
    print("Сначала выполним код, написанный в функции a, а потом:")
    callback()
    
def b():
    print("код в функции b")

def c():
    print("код в функции c")
    
a(b)

In [ ]:
a(c)

## Создание своего декоратора

В терминах функций в питоне **декоратор** - это "обертка" над декорируемой функцией. Т.е. это по сути новая функция, которая выполняет какой-то код до выполнения исходной функции, затем выполняет саму исходную функцию, и в конце выполняет еще какой-то код. Напишем свой простейший декоратор.

In [ ]:
def decorator(source_func):
    # вот та самая функция, которая выполняет некоторый код до и после исходной функции
    def wrapper():
        print("Код ДО исходной функции")
        source_func()
        print("Код ПОСЛЕ исходной функции")
    
    # нам может понадобиться декорировать несколько разных функций, поэтому внешняя функция decorator возвращает
    # новую функцию, принимая в аргументах исходную
    return wrapper

Если мы хотим переопределить поведение какой-то функции, не внося в нее изменения, то мы просто должны присвоить переменной, ссылающейся на исходную функцию, ссылку на новую, обернутую, функцию.

In [ ]:
def func():
    print("код ВНУТРИ исходной функции")
    
func()

In [ ]:
decorator(func)

In [ ]:
decorator(func)()

In [ ]:
func = decorator(func)

func()

А что если выполнить код `func = decorator(func)` еще раз?

In [ ]:
func = decorator(func)

func()

## Синтаксический сахар

Еще раз определим функцию `func` и обернем ее в декоратор _in a pythonic way_

In [ ]:
@decorator
def func():
    print("код ВНУТРИ исходной функции")
    
func()

Таким образом, запись в предыдущей ячейке эквивалентна следующему:

In [ ]:
def func():
    print("код ВНУТРИ исходной функции")
    
func = decorator(func)

Кстати, в этом виде декоратор можно также применять несколько раз:

In [ ]:
@decorator
@decorator
def func():
    print("код ВНУТРИ исходной функции")
    
func()

Также мы можем обернуть функцию в несколько декораторов. Порядок оборачивания - важен.

In [ ]:
def decorator1(func):
    def wrapper():
        print("Декоратор 1 - начало")
        func()
        print("Декоратор 1 - конец")
    return wrapper


def decorator2(func):
    def wrapper():
        print("Декоратор 2 - начало")
        func()
        print("Декоратор 2 - конец")
    return wrapper

In [ ]:
@decorator1
@decorator2
def f():
    print("Функция")
    
f()

In [ ]:
@decorator2
@decorator1
def f():
    print("Функция")
    
f()

## Зачем нужны декораторы, если и так можно из одной функции вызывать другие?

Рассмотрим пример из проекта Django - фреймворка для быстрого и удобного создания серверных приложений.

Допустим, у нас на сервере есть несколько адресов, доступ к которым должен быть открыт только зарегистрированным пользователям. Механизм регистрации и авторизации пользователей реализован где-то в недрах джанги, мы его не должны трогать. Поэтому для того, чтобы перед исполнением функции проверить, зарегистрирован ли пользователь, Django предоставляет нам декоратор: https://docs.djangoproject.com/en/3.0/topics/auth/default/#the-login-required-decorator

In [ ]:
from django.contrib.auth.decorators import login_required

@login_required
def my_view(request):
    pass

## Задание:

Написать декоратор, который после выполнения функции выводит на экран время, которое было затрачено на выполнение. При этом нужно учесть вариант, когда оборачиваемая функция возвращает какое-то значение =)

# Проброс аргументов в декорируемую функцию

Ведь какой смысл в функциях без аргументов?

Допустим, мы хотим обернуть функцию, которая принимает 2 аргумента:

In [ ]:
def func_with_2_args(name, last_name):
    print("Здесь начинает выполоняться функция")
    print("Имя:", name)
    print("Фамилия:", last_name)
    
func_with_2_args("Евпатий", "Коловрат")

Тогда нам нужно создать такую функцию-обертку, которая тоже будет принимать 2 аргумента, чтобы не поломать интерфейс взаимодействия с исходной функцией.

In [ ]:
def decorator_with_2_args(func):
    def wrapper_with_2_args(arg1, arg2):
        print("Аргументы, которые получил декоратор:", ", ".join((arg1, arg2)))
        func(arg1, arg2)
    return wrapper_with_2_args

@decorator_with_2_args
def func_with_2_args(name, last_name):
    print("Здесь начинает выполоняться функция")
    print("Имя:", name)
    print("Фамилия:", last_name)

func_with_2_args("Евпатий", "Коловрат")

### Декорирование методов

Теперь, когда мы научились пробрасывать аргументы в функцию, мы можем оборачивать и методы классов. Ведь единственное, что отличает метод класса от обычной функции - это то, что при его вызове первым аргументом передается `self`

In [ ]:
def method_decorator(method):
    def method_wrapper(self, x):
        return method(self, x * 1.15) # возьмем комиссию за операцию
    return method_wrapper


class Price:
    def __init__(self, price):
        self.price = price            # назначим цену
        
    @method_decorator
    def get_price(self, exchange_rate):
        return self.price * exchange_rate   # вернем цену в другой валюте. А в декораторе мы описали комиссию за операцию =)
    
p = Price(100)
p.get_price(72)

### Общий вид декораторов с аргументами

Конечно, мы хотим применять декораторы к разным функциям, которые принимают разное количество аргументов. Для этого можем воспользоваться `*args, **kwargs`

In [ ]:
def common_decorator(func):
    def common_wrapper(*args, **kwargs):
        print("Декоратор принял args:", args)
        print("Декоратор принял kwargs:", kwargs)
        result = func(*args, **kwargs)
        print("Декоратор завершается\n")
        return result
    return common_wrapper

@common_decorator
def zero_args():
    print("Функция без аргументов")
    
@common_decorator
def one_arg(x):
    print("Функция с одним аргументом х =", x)
    
@common_decorator
def two_args(x, y):
    print("Функция с двумя аргументами x, y =", x, y)
    
@common_decorator
def multiple_args(*args):
    print("Функция, которая приняла args:", args)
    
@common_decorator
def any_arg(*args, **kwargs):
    print("Функция, которая приняла args:", args, "и kwargs:", kwargs)
    
    
zero_args()
one_arg(25)
two_args("ПРИВЕТ", "АРГУМЕНТ")
multiple_args(1, 2, 3, 4, 5)
any_arg(1, 2, 3, 4, a="A", b="B", e=2.71)

## Аргументы декораторов

А что если нужно оборачивать функцию по-разному в зависимости от каких-то внешних условий? Например, задание URL-адресов в фреймворке flask реализовано так: https://flask.palletsprojects.com/en/1.1.x/quickstart/

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

Давайте еще раз зафиксируем то, что синтаксис

`@decorator`

просто вызывает функцию `decorator` и присваивает результат выполнения в переменную. А давайте сейчас создадим обертку над декоратором!

In [ ]:
def decorator_maker():
    print("decorator_maker started")
    def decorator(func):
        print("decorator started")
        def wrapper():
            print("wrapper started")
            func()
            print("wrapper ended")
        print("decorator returns wrapper")
        return wrapper
    print("decorator_maker returns decorator")
    return decorator

def usual_func():
    print("usual function")

Запустим всё это по частям

In [ ]:
new_decorator = decorator_maker()

In [ ]:
usual_func = new_decorator(usual_func)

In [ ]:
usual_func()

А теперь запишем то же самое через @:

In [ ]:
@decorator_maker()       # вызвали функцию decorator_maker и применили @ уже к возвращенному ей декоратору
def usual_func():
    print("usual function")

In [ ]:
usual_func()

### А теперь добавим аргументы в получившуюся конструкцию

In [ ]:
def decorator_maker(dec_arg_1, dec_arg_2):
    print("Inside decorator_maker. Decorator args:", dec_arg_1, dec_arg_2)
    def decorator(func):
        print("Inside decorator. Decorator args:", dec_arg_1, dec_arg_2)
        def wrapper(func_arg_1, func_arg_2):
            print("Inside wrapper. Decorator args:", dec_arg_1, dec_arg_2)
            print("Inside wrapper. Function args:", func_arg_1, func_arg_2)
        return wrapper
    return decorator

In [ ]:
@decorator_maker("DEC_ARG_1", "DEC_ARG_2")
def func(s1, s2):
    print("Inside function. Function args:", s1, s2)

In [ ]:
func("FUNC_ARG_1", "FUNC_ARG_2")

## Поле `__name__` и библиотка functools

У каждого объекта-фукнции мы можем получить строковое представление ее имени через поле `__name__`

In [ ]:
def our_function():
    pass

our_function.__name__

А меняет ли это свойство декоратор?

In [ ]:
@decorator
def decorated_function():
    pass

decorated_function.__name__

### functools

В питоне есть библиотека functools, в которой реализованы некоторые функции для работы с вызываемыми объектами. Например, в ней есть реализация декоратора, который задает аргументы по умолчанию для функции:

In [ ]:
from functools import partial

print(int('10010'))
print(int('10010', base=2), end='\n\n')

basetwo = partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int.'
print(basetwo('10010'))

Или, например, в Python 3.8 есть декоратор `@cached_property`, который позволяет не тратить ресурсы на пересчет property каждый раз при вызове, а сохраняет значение после первого вычисления. https://docs.python.org/3.8/library/functools.html

In [ ]:
# у нас установлен Python 3.6, поэтому этот код не сработает

class DataSet:
    def __init__(self, sequence_of_numbers):
        self._data = sequence_of_numbers

    @cached_property
    def stdev(self):
        return statistics.stdev(self._data)

    @cached_property
    def variance(self):
        return statistics.variance(self._data)

### functools.wraps

Но тем не менее мы обратились к этой библиотеке для того, чтобы сохранять исходные имена оборачиваемых функций. Посмотрим, как это делается.

In [ ]:
from functools import wraps


def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper


@my_decorator
def example():
    """Docstring"""
    print('Called example function')
    
example()

In [ ]:
example.__name__

## Задание

Написать логгер вызова функций. Функции оборачиваются в декораторы, в которые передается название логгера для записи. Сами логгеры можно реализовать списками словарей. В логах должно быть отмечено название вызываемой функции, параметры, с которыми она вызывается, время вызова и возвращаемое значение.

# Альтернативное применение декораторов

Поскольку декораторы - это просто синтаксический сахар, который вызывает функции, можно придумать менее стандартное поведение для вызываемых функций.

## Регистрация функций / событий

Например, нам нужно вызвать целый ряд функций, когда происходит какое-то событие. Для этого их надо где-то хранить и динамически добавлять при создании. Для этого можно использовать декораторы. (https://medium.com/better-programming/decorators-in-python-72a1d578eac4)

In [ ]:
PLUGINS = dict()

# это наш декоратор, в котором мы регистрируем функцию
def register(func):
    PLUGINS[func.__name__] = func
    return func

@register
def add(a, b):
    return a+b

@register
def multiply(a, b):
    return a*b

def operation(func_name, a, b):
    func = PLUGINS[func_name]
    return func(a, b)

print(PLUGINS)
print(operation('add', 2, 3))
print(operation('multiply', 2, 3))

## Обертки над классами

В питоновском классе есть два метода, которые совершают вызов объекта: `__init__()` и `__call__()`. Обернув класс в декоратор, мы можем поколдовать над этими методами.

In [ ]:
@common_decorator
class Calculator:

    def __init__(self, num):
        self.num = num
        import time
        time.sleep(2)
        
    def __call__(self):
        print("call")

    def doubled_and_add(self):
        res = sum([i * 2 for i in range(self.num)])
        print("Result : {}".format(res))

c = Calculator(100)

In [ ]:
c.doubled_and_add()

In [ ]:
common_decorator(c)()